In [44]:
import pandas as pd
import csv

In [45]:
headers = []
identifiers = []
data = []

with open('Endurance2016.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    
    for row in csv_reader:
        if line_count == 0:
            headers.append(row[0])
#             print(f'Column names are {", ".join(row)}')
            line_count += 1
        elif row[0].isdigit():
#             print(f'{row[0]}')
            data.append(row)
            line_count += 1
        else:
            identifiers.append(row[0])
    print(f'Processed {line_count} lines.')

print(headers)
print('--------')
print(len(identifiers), identifiers[0:10])
print('--------')
print(len(data), data[0:10])

Processed 1381260 lines.
['PER CSV Modbus Log 6/18/2016 1:29:12 PM']
--------
263 ['Value PCM Fault Watchdog: 0', 'Value PCM Fault Brownout: 1', 'Value PCM Fault Can Fault: 2', 'Value PCM Fault Moc Timeout: 3', 'Value PCM Fault Coolant Nflow: 4', 'Value PCM Mode Startup: 5', 'Value PCM Mode Standby: 6', 'Value PCM Mode Idle: 7', 'Value PCM Mode Ready: 8', 'Value PCM Mode Error: 9']
--------
1381259 [['4', '227', '1'], ['4', '228', '0'], ['4', '229', '0'], ['4', '231', '1'], ['4', '232', '0'], ['4', '233', '1'], ['4', '234', '0'], ['5', '235', '0'], ['5', '236', '0'], ['5', '237', '0']]


# Cleaning

In [46]:
identifier_ids = []
good_values = ["AMS Voltage", "AMS Current", "PCM Front MPH 0", "PCM Front MPH 1", 
               "PCM Rear MPH 0", "PCM Rear MPH 1"]
identifiers = [identifier for identifier in identifiers if any(val in identifier for val in good_values)]
print(identifiers)

#identifier_ids.append(identifier.split(': ')[1])


['Value PCM Front MPH 0: 95', 'Value PCM Front MPH 1: 96', 'Value PCM Rear MPH 0: 97', 'Value PCM Rear MPH 1: 98', 'Value AMS Current: 170', 'Value AMS Voltage: 172']


In [48]:
identifier_data = []
for i in range(len(identifiers)):
    identifier_data_temp = []
 
    identifier_data_temp.append(identifiers[i])
    for j in range(len(data)):
        if identifiers[i].split(': ')[1] == data[j][1]:

            identifier_data_temp.append(data[j])
    identifier_data.append(identifier_data_temp)
print(len(identifier_data), identifier_data[0])

6 ['Value PCM Front MPH 0: 95', ['62', '95', '30.51657'], ['1072', '95', '25.2791'], ['1176', '95', '24.00267'], ['1298', '95', '23.37095'], ['1381', '95', '23.23142'], ['1577', '95', '23.42241'], ['1578', '95', '23.42241'], ['1578', '95', '23.42241'], ['1798', '95', '24.28916'], ['1882', '95', '23.9088'], ['1996', '95', '23.89442'], ['2346', '95', '21.9668'], ['2584', '95', '19.01742'], ['2705', '95', '18.50047'], ['2899', '95', '18.19015'], ['3292', '95', '18.78601'], ['3606', '95', '20.68851'], ['3801', '95', '22.18554'], ['4422', '95', '24.64035'], ['4844', '95', '25.8881'], ['4904', '95', '25.86024'], ['5017', '95', '25.50095'], ['5550', '95', '22.72947'], ['5550', '95', '22.72947'], ['5715', '95', '21.79372'], ['6025', '95', '20.65325'], ['6231', '95', '21.7398'], ['6232', '95', '21.7398'], ['6627', '95', '24.42209'], ['6846', '95', '24.21928'], ['6936', '95', '24.05374'], ['6936', '95', '24.05374'], ['7171', '95', '23.75805'], ['7226', '95', '24.11085'], ['7445', '95', '24.72896

In [68]:
pcm_front_0 = [float(val[2]) for val in identifier_data[0][1:]]
pcm_front_1 = [float(val[2]) for val in identifier_data[1][1:]]
pcm_rear_0 = [float(val[2]) for val in identifier_data[2][1:]]
pcm_rear_1 = [float(val[2]) for val in identifier_data[3][1:]]
print(max(pcm_front_0), max(pcm_front_1), max(pcm_rear_0), max(pcm_rear_1))
print(min(pcm_front_0), min(pcm_front_1), min(pcm_rear_0), min(pcm_rear_1))

63.83601 65.26362 67.53808 67.6329
0.0 0.0 0.0 0.0


Note: Car stops near the end, I count it in

In [74]:
total_distance = 0.0
total_distance += float(identifier_data[0][1][0]) / 3600000 *  float(identifier_data[0][1][2])
for i in range(2, len(identifier_data[0])):
    total_distance += (float(identifier_data[0][i][0]) - float(identifier_data[0][i-1][0])) / 3600000 *  float(identifier_data[0][i][2])
    # integrate with reinman sum
print(total_distance / (float(identifier_data[0][-1][0]) / 3600000))

20.241668003988906


In [132]:
powers = []

print(len(identifier_data[4]), len(identifier_data[5]), identifier_data[4][-1][0], identifier_data[5][-1][0])

common_times = list(set([value[0] for value in identifier_data[4][1:] if value[0] in [val[0] for val in identifier_data[5][1:]]]))
current_times = []
for i in range(1, len(identifier_data[4])):
    if identifier_data[4][i][0] in common_times and identifier_data[4][i][0] not in [val[0] for val in current_times]:
        current_times.append(identifier_data[4][i])

voltage_times = []
for i in range(1, len(identifier_data[5])):
    if identifier_data[5][i][0] in common_times and identifier_data[5][i][0] not in [val[0] for val in voltage_times]:
        voltage_times.append(identifier_data[5][i])
print(len(current_times), len(voltage_times))


for i in range(len(current_times)):
    powers.append([float(current_times[i][0]), float(current_times[i][2]) * float(voltage_times[i][2])])

print(powers[0:10])

5518 5876 1176857 1176857
4826 4826
[[239.0, 2977.951520688], [425.0, 17938.810802925], [543.0, -6768.783638661], [713.0, -6598.1333821709995], [857.0, -10147.739602484002], [921.0, -10152.584266638], [1019.0, -9940.928960232], [1205.0, -9674.872270578], [1298.0, -1071.5838825612], [1405.0, 2513.6125088032]]


Sometimes times don't line up for 4th (current) and 5th (voltage) columns. Also remove duplicates

In [141]:
total_energy = 0.0
total_energy += powers[0][0] / 3600000 *  powers[0][1] / 1000
for i in range(2, len(powers)):
    total_energy += (powers[i][0] - powers[i-1][0]) / 3600000 *  powers[i][1] / 1000
    # integrate with reinman sum
print(total_energy)

1.9924271932654989
